# SegmentNT Binary Classifier for DNA Breakpoint Detection

This notebook implements a complete pipeline to fine-tune SegmentNT for classifying DNA sequences as positive or negative breakpoints, similar to FusionAI.

## Overview

**What this notebook does:**
1. **Prepares data** from FusionAI-format CSV files (positive/negative breakpoints)
2. **Merges sequences** (5' + 3' = 20kb input)
3. **Fine-tunes SegmentNT** with a binary classification head
4. **Makes predictions** on test data

**Key Features:**
- Uses SegmentNT with Nucleotide Transformer backbone (500M parameters)
- Handles sequences up to 30kb (can extend to 50kb)
- 6-mer tokenization for efficient DNA encoding
- Comprehensive metrics (Accuracy, Precision, Recall, F1, AUC)

**About SegmentNT:**
- Segmentation model built on Nucleotide Transformer
- Trained on 14 different genomic element classes
- 562M total parameters (500M backbone + 62M segmentation head)
- Single-nucleotide resolution predictions

---

## 📦 Installation & Setup

First, let's install all required dependencies and check our compute environment.

**Note:** SegmentNT requires transformers to be installed from source until the next official release.

In [1]:
# Install required packages
# SegmentNT requires transformers from source
# !pip install -q --upgrade git+https://github.com/huggingface/transformers.git
# !pip install -q torch accelerate pandas scikit-learn

print("✅ Installation complete!")

✅ Installation complete!


In [2]:
# Check GPU availability and setup
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: No GPU detected! Training will be very slow.")
    print("   Consider using a GPU runtime (Runtime → Change runtime type → GPU)")

PyTorch version: 2.9.1+cu128
CUDA available: True
GPU Device: NVIDIA A100 80GB PCIe
GPU Memory: 84.93 GB


/home/akp1/GeneAI/venv/lib/python3.12/site-packages/torch/cuda/__init__.py:827: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## 📚 Import Libraries

Import all necessary libraries for data processing, model training, and evaluation.

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer, AutoConfig
from safetensors.torch import load_model, save_model
from typing import Dict, List, Tuple
import os
from tqdm.auto import tqdm
import warnings

warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
print("✅ Libraries imported successfully!")

/home/akp1/GeneAI/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Libraries imported successfully!


## ⚙️ Configuration

Set all hyperparameters and file paths here. **Update the CSV paths to match your data!**

### SegmentNT Model Info:
- **Model:** `InstaDeepAI/segment_nt`
- **Max sequence length:** 30,000 bp (can extend to 50kb with rescaling)
- **Tokenization:** 6-mer tokens (vocabulary size: 4105)
- **Parameters:** 562M (500M backbone + 62M segmentation head)
- **Architecture:** Nucleotide Transformer + U-Net segmentation head

### Important Notes:
- The number of DNA tokens (excluding CLS token) must be divisible by 4
- For sequences between 30kb-50kb, rescaling_factor needs adjustment
- Default training length: 30,000 nucleotides (5001 tokens including CLS)

In [4]:
# ============ FILE PATHS - UPDATE THESE! ============
POSITIVE_CSV = "datasets/fusion_gene_positive_bp_information_with_class_for_modeling.txt"  # Path to positive breakpoint CSV
NEGATIVE_CSV = "datasets/fusion_gene_negative_bp_information_with_class_for_modeling.txt"  # Path to negative breakpoint CSV
TEST_CSV = "datasets/fusion_gene_positive_bp_information_with_class_for_testing.txt"          # Path to test data CSV
COLUMNS = ["Hgene","Hchr","Hbp","Hstrand","Tgene","Tchr","Tbp","Tstrand","5'-gene sequence (10Kb)","3'-gene sequence (10Kb)"] # CSV Columns names
OUTPUT_DIR = "./segmentnt_breakpoint_model" # Where to save the model

# ============ MODEL CONFIGURATION ============
MODEL_NAME = "InstaDeepAI/segment_nt"  # SegmentNT model
MAX_LENGTH = 20000  # Maximum sequence length (20kb - within 30kb limit)

# Note: MAX_LENGTH must satisfy: (MAX_LENGTH_IN_TOKENS - 1) % 4 == 0
# For 20kb sequences, this gives us approximately 3334 tokens
# We'll handle this in the tokenizer

# ============ TRAINING HYPERPARAMETERS ============
NUM_EPOCHS = 3
BATCH_SIZE = 4          # SegmentNT is larger (562M params), so reduce batch size
LEARNING_RATE = 2e-5    # Learning rate
WARMUP_STEPS = 500      # Learning rate warmup
WEIGHT_DECAY = 0.01     # L2 regularization
VAL_SPLIT = 0.2         # Validation split (20%)

# ============ DEVICE SETUP ============
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"Model: {MODEL_NAME}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Training epochs: {NUM_EPOCHS}")
print(f"Max sequence length: {MAX_LENGTH} bp")

Using device: cuda
Model: InstaDeepAI/segment_nt
Batch size: 4
Training epochs: 3
Max sequence length: 20000 bp


## 📊 Step 1: Data Preparation Class

This class handles loading and preparing the FusionAI CSV files.

**Input CSV Format (tab-separated):**
- `Hgene`, `Hchr`, `Hbp`, `Hstrand` - Head gene information
- `Tgene`, `Tchr`, `Tbp`, `Tstrand` - Tail gene information
- `5'-gene sequence (10Kb)` - 5' sequence (10,000 bp)
- `3'-gene sequence (10Kb)` - 3' sequence (10,000 bp)

**What it does:**
1. Loads positive and negative CSV files
2. Merges 5' + 3' sequences → 20kb sequences
3. Adds labels (1=positive, 0=negative)
4. Combines and shuffles data
5. Validates sequences

In [5]:
class BreakpointDataPreparation:
    """
    Handles loading and preparing breakpoint data from CSV files.
    
    This class reads FusionAI-format CSV files containing positive and negative
    breakpoint examples, merges the 5' and 3' sequences, and prepares the data
    for model training.
    """
    
    def __init__(self, columns: List[str]):
        """
        Initialize the data preparation class.
        
        Args:
            columns: List of column names for the CSV files
        """
        self.columns = columns
        self.seq_5_col = "5'-gene sequence (10Kb)"
        self.seq_3_col = "3'-gene sequence (10Kb)"
    
    def load_csv(self, filepath: str, label: int) -> pd.DataFrame:
        """
        Load a CSV file and add label column.
        
        Args:
            filepath: Path to the CSV file
            label: Label to assign (1 for positive, 0 for negative)
            
        Returns:
            DataFrame with loaded data and label column
        """
        print(f"Loading {filepath}...")
        df = pd.read_csv(filepath, sep='\t', names=self.columns)
        df['label'] = label
        print(f"  Loaded {len(df)} samples")
        return df
    
    def merge_sequences(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Merge 5' and 3' sequences into a single combined sequence.
        
        Args:
            df: DataFrame containing 5' and 3' sequence columns
            
        Returns:
            DataFrame with added 'merged_sequence' column
        """
        print("Merging 5' and 3' sequences...")
        df['merged_sequence'] = df[self.seq_5_col] + df[self.seq_3_col]
        
        # Validate sequences
        invalid_chars = df['merged_sequence'].str.contains('[^ACGT]', regex=True, na=False)
        if invalid_chars.any():
            print(f"  ⚠️ Warning: Found {invalid_chars.sum()} sequences with non-ACGT characters")
            print("     These will be handled by the tokenizer")
        
        # Check sequence lengths
        seq_lengths = df['merged_sequence'].str.len()
        print(f"  Sequence length stats:")
        print(f"    Min: {seq_lengths.min()} bp")
        print(f"    Max: {seq_lengths.max()} bp")
        print(f"    Mean: {seq_lengths.mean():.0f} bp")
        
        return df
    
    def prepare_data(self, positive_csv: str, negative_csv: str) -> pd.DataFrame:
        """
        Load and prepare the complete dataset.
        
        Args:
            positive_csv: Path to positive samples CSV
            negative_csv: Path to negative samples CSV
            
        Returns:
            Combined and shuffled DataFrame
        """
        print("="*70)
        print("PREPARING TRAINING DATA")
        print("="*70)
        
        # Load datasets
        df_positive = self.load_csv(positive_csv, label=1)
        df_negative = self.load_csv(negative_csv, label=0)
        
        # Merge sequences
        df_positive = self.merge_sequences(df_positive)
        df_negative = self.merge_sequences(df_negative)
        
        # Combine datasets
        print("\nCombining datasets...")
        df_combined = pd.concat([df_positive, df_negative], ignore_index=True)
        
        # Shuffle
        df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)
        
        print(f"\n✅ Data preparation complete!")
        print(f"   Total samples: {len(df_combined)}")
        print(f"   Positive samples: {(df_combined['label'] == 1).sum()} ({(df_combined['label'] == 1).sum() / len(df_combined) * 100:.1f}%)")
        print(f"   Negative samples: {(df_combined['label'] == 0).sum()} ({(df_combined['label'] == 0).sum() / len(df_combined) * 100:.1f}%)")
        
        return df_combined

## 🧬 Step 2: Custom Dataset Class

PyTorch Dataset class for DNA sequences. This handles:
- DNA sequence tokenization using SegmentNT's 6-mer tokenizer
- Proper padding and truncation
- Attention mask generation
- Label formatting

In [6]:
class DNASequenceDataset(Dataset):
    """
    PyTorch Dataset for DNA sequences.
    
    Handles tokenization of DNA sequences using SegmentNT's Nucleotide Transformer
    tokenizer, which uses 6-mer tokenization.
    """
    
    def __init__(self, sequences: List[str], labels: List[int], tokenizer, max_length: int):
        """
        Initialize the dataset.
        
        Args:
            sequences: List of DNA sequences
            labels: List of labels (0 or 1)
            tokenizer: SegmentNT tokenizer
            max_length: Maximum token length (not base pairs)
        """
        self.sequences = sequences
        self.labels = labels
        self.tokenizer = tokenizer
        
        # Calculate token length that satisfies (token_length - 1) % 4 == 0
        # SegmentNT requires this for its downsampling blocks
        # For 20kb sequences (~3333 tokens), we use 3333 tokens
        base_tokens = int(max_length / 6)  # Approximate tokens for 6-mer tokenization
        # Find nearest valid token length
        while (base_tokens - 1) % 4 != 0:
            base_tokens += 1
        
        self.max_token_length = base_tokens
        print(f"Using max token length: {self.max_token_length} (for ~{max_length} bp sequences)")
    
    def __len__(self) -> int:
        return len(self.sequences)
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        """
        Get a single item from the dataset.
        
        Args:
            idx: Index of the item
            
        Returns:
            Dictionary containing input_ids, attention_mask, and labels
        """
        sequence = self.sequences[idx]
        label = self.labels[idx]
        
        # Tokenize sequence
        encoding = self.tokenizer(
            sequence,
            add_special_tokens=True,
            max_length=self.max_token_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

## 🏗️ Step 3: SegmentNT Classification Model

Wraps the SegmentNT model with a binary classification head.

**Architecture:**
1. **SegmentNT Encoder** (500M params) - Nucleotide Transformer backbone
2. **U-Net Segmentation Head** (62M params) - Original SegmentNT head
3. **Classification Head** - Our custom binary classifier
   - Global average pooling over sequence
   - Dropout (0.1)
   - Linear layer → 2 classes

**Note:** We'll use the hidden states from SegmentNT and add our own classification head.

In [7]:
class SegmentNTForBinaryClassification(nn.Module):
    """
    SegmentNT model with a binary classification head.
    
    Takes the SegmentNT encoder and adds a classification head on top
    for binary classification of DNA sequences.
    """
    
    def __init__(self, model_name: str, num_labels: int = 2, dropout: float = 0.1):
        """
        Initialize the classification model.
        
        Args:
            model_name: Name of the SegmentNT model
            num_labels: Number of output classes (2 for binary classification)
            dropout: Dropout probability
        """
        super().__init__()
        
        print(f"Loading SegmentNT model: {model_name}...")
        # config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
        # print(f"Intermediate size: {getattr(config, 'intermediate_size', 'NOT SET')}")

        self.segmentnt = AutoModel.from_pretrained(
            model_name,
            trust_remote_code=True,
            ignore_mismatched_sizes=True  # We want to catch mismatches
        )
        
        # Get hidden size from the model config
        self.hidden_size = self.segmentnt.hidden_size
        print(f"  Hidden size: {self.hidden_size}")
        print(f"  Model loaded successfully!")
        
        # Classification head
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.hidden_size, num_labels)
        
        print(f"  Added classification head: {self.hidden_size} -> {num_labels}")
    
    def forward(self, input_ids, attention_mask=None, labels=None):
        """
        Forward pass through the model.
        
        Args:
            input_ids: Tokenized input sequences
            attention_mask: Attention mask for padded tokens
            labels: Ground truth labels (optional)
            
        Returns:
            Dictionary containing loss (if labels provided) and logits
        """
        # Get SegmentNT outputs
        outputs = self.segmentnt(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )
        
        # Use the last hidden state
        # Shape: (batch_size, sequence_length, hidden_size)
        hidden_states = outputs.hidden_states[-1]
        
        # Global average pooling over the sequence dimension
        # Only pool over non-padded tokens using attention mask
        if attention_mask is not None:
            # Expand attention mask to match hidden states shape
            mask = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
            sum_hidden = torch.sum(hidden_states * mask, dim=1)
            sum_mask = torch.sum(mask, dim=1)
            pooled_output = sum_hidden / sum_mask
        else:
            pooled_output = torch.mean(hidden_states, dim=1)
        
        # Apply dropout and classifier
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        
        # Calculate loss if labels are provided
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
        
        return {
            'loss': loss,
            'logits': logits
        }

## 📈 Step 4: Evaluation Metrics

Function to compute comprehensive classification metrics.

**Metrics computed:**
- **Accuracy** - Overall correctness
- **Precision** - Of predicted positives, how many are correct?
- **Recall** - Of actual positives, how many did we find?
- **F1 Score** - Harmonic mean of precision and recall
- **AUC-ROC** - Area under the ROC curve (model discrimination ability)

In [8]:
def compute_metrics(eval_pred):
    """
    Compute evaluation metrics for the model.
    
    Args:
        eval_pred: Tuple of (predictions, labels)
        
    Returns:
        Dictionary of metrics
    """
    logits, labels = eval_pred
    
    # Convert logits to predictions
    predictions = np.argmax(logits, axis=-1)
    
    # Calculate metrics
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary'
    )
    
    # Calculate AUC-ROC
    probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()[:, 1]
    auc = roc_auc_score(labels, probs)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc
    }

## 🚀 Step 5: Complete Training Pipeline

This class orchestrates the entire workflow:
1. Data preparation
2. Model initialization
3. Training with Hugging Face Trainer
4. Evaluation
5. Model saving
6. Predictions

**Key Features:**
- Automatic train/validation split
- FP16 mixed precision training
- Gradient accumulation
- Learning rate scheduling with warmup
- Checkpoint saving

In [9]:
class SegmentNTTrainingPipeline:
    """
    Complete pipeline for training SegmentNT on breakpoint classification.
    """
    
    def __init__(self, model_name: str, output_dir: str, max_length: int):
        """
        Initialize the training pipeline.
        
        Args:
            model_name: Name of the SegmentNT model
            output_dir: Directory to save model and outputs
            max_length: Maximum sequence length in base pairs
        """
        self.model_name = model_name
        self.output_dir = output_dir
        self.max_length = max_length
        
        # Load tokenizer
        print(f"Loading SegmentNT tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name, 
            trust_remote_code=True
        )
        print(f"  Tokenizer loaded!")
        print(f"  Vocabulary size: {len(self.tokenizer)}")
        print(f"  Tokenization: 6-mer based")
        
        self.model = None
        self.trainer = None
    
    def prepare_datasets(self, df: pd.DataFrame, val_split: float = 0.2):
        """
        Prepare training and validation datasets.
        
        Args:
            df: DataFrame with 'merged_sequence' and 'label' columns
            val_split: Fraction of data to use for validation
            
        Returns:
            Tuple of (train_dataset, val_dataset)
        """
        print("\n" + "="*70)
        print("PREPARING DATASETS")
        print("="*70)
        
        # Split data
        train_df, val_df = train_test_split(
            df, test_size=val_split, random_state=42, stratify=df['label']
        )
        
        print(f"Training samples: {len(train_df)}")
        print(f"Validation samples: {len(val_df)}")
        
        # Create datasets
        train_dataset = DNASequenceDataset(
            sequences=train_df['merged_sequence'].tolist(),
            labels=train_df['label'].tolist(),
            tokenizer=self.tokenizer,
            max_length=self.max_length
        )
        
        val_dataset = DNASequenceDataset(
            sequences=val_df['merged_sequence'].tolist(),
            labels=val_df['label'].tolist(),
            tokenizer=self.tokenizer,
            max_length=self.max_length
        )
        
        print("\n✅ Datasets prepared!")
        return train_dataset, val_dataset
    
    def train(self, train_dataset, val_dataset, num_epochs: int, 
              batch_size: int, learning_rate: float, warmup_steps: int,
              weight_decay: float):
        """
        Train the model.
        
        Args:
            train_dataset: Training dataset
            val_dataset: Validation dataset
            num_epochs: Number of training epochs
            batch_size: Training batch size
            learning_rate: Learning rate
            warmup_steps: Number of warmup steps
            weight_decay: L2 regularization weight
        """
        print("\n" + "="*70)
        print("INITIALIZING MODEL")
        print("="*70)
        
        # Initialize model
        self.model = SegmentNTForBinaryClassification(self.model_name)
        self.model.to(device)
        
        print("\n" + "="*70)
        print("TRAINING CONFIGURATION")
        print("="*70)
        print(f"Epochs: {num_epochs}")
        print(f"Batch size: {batch_size}")
        print(f"Learning rate: {learning_rate}")
        print(f"Warmup steps: {warmup_steps}")
        print(f"Weight decay: {weight_decay}")
        use_bf16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
        # Training arguments
        training_args = TrainingArguments(
             output_dir=self.output_dir,
            num_train_epochs=num_epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            learning_rate=learning_rate,
            warmup_steps=warmup_steps,
            weight_decay=weight_decay,        
            max_grad_norm=1.0,             
            lr_scheduler_type="cosine",
            bf16=use_bf16,
            fp16=False, 
            logging_dir=f'{self.output_dir}/logs',
            logging_steps=50,
            eval_strategy="steps",
            eval_steps=500,
            save_strategy="steps",
            save_steps=500,
            save_total_limit=3,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            dataloader_num_workers=4,
            remove_unused_columns=False,
            report_to="none",
            save_safetensors=False,
        )
        
        # Initialize trainer
        self.trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics,
        )
        
        print("\n" + "="*70)
        print("STARTING TRAINING")
        print("="*70)
        
        # Train
        self.trainer.train()
        
        print("\n" + "="*70)
        print("TRAINING COMPLETE!")
        print("="*70)
    
    def evaluate(self):
        """
        Evaluate the trained model on validation data.
        
        Returns:
            Dictionary of evaluation metrics
        """
        print("\n" + "="*70)
        print("EVALUATING MODEL")
        print("="*70)
        
        metrics = self.trainer.evaluate()
        
        print("\nValidation Metrics:")
        print(f"  Accuracy:  {metrics['eval_accuracy']:.4f}")
        print(f"  Precision: {metrics['eval_precision']:.4f}")
        print(f"  Recall:    {metrics['eval_recall']:.4f}")
        print(f"  F1 Score:  {metrics['eval_f1']:.4f}")
        print(f"  AUC-ROC:   {metrics['eval_auc']:.4f}")
        
        return metrics
    
    def save_model(self, save_path: str = None):
        """
        Save the trained model.
        
        Args:
            save_path: Path to save the model (default: output_dir/final_model)
        """
        if save_path is None:
            save_path = os.path.join(self.output_dir, 'final_model')
        
        print(f"\nSaving model to {save_path}...")
        save_model(self.model, f"{self.output_dir}/final_model/model.safetensors")
        self.tokenizer.save_pretrained(f"{self.output_dir}/final_model")
        print("✅ Model saved!")
    
    def predict(self, test_csv_path: str, model_path: str = None, 
                output_path: str = "predictions.csv", batch_size: int = 8) -> pd.DataFrame:
        """
        Make predictions on test data.
        
        Args:
            test_csv_path: Path to test CSV file
            model_path: Path to trained model (default: use current model)
            output_path: Path to save predictions CSV
            batch_size: Batch size for inference
            
        Returns:
            DataFrame with predictions
        """
        print("\n" + "="*70)
        print("MAKING PREDICTIONS")
        print("="*70)
        
        # Load test data
        print(f"Loading test data from {test_csv_path}...")
        df_test = pd.read_csv(test_csv_path, sep='\t', names=COLUMNS)
        df_test['merged_sequence'] = df_test["5'-gene sequence (10Kb)"] + df_test["3'-gene sequence (10Kb)"]
        print(f"  Loaded {len(df_test)} test samples")
        
        # Load model if path provided
        if self.model == None:
            print(f"\nLoading model from {model_path}...")
            self.model = HyenaDNAClassifier(self.model_name, num_labels=2)
            load_model(self.model, f"{model_path}/model.safetensors")
            self.model.to(self.device)
            self.tokenizer = AutoTokenizer.from_pretrained(f"{OUTPUT_DIR}/final_model",
            trust_remote_code=True)
        self.model.eval()
        
        # Create test dataset
        test_dataset = DNASequenceDataset(
            sequences=df_test['merged_sequence'].tolist(),
            labels=[0] * len(df_test),  # Dummy labels
            tokenizer=self.tokenizer,
            max_length=self.max_length
        )
        
        # Make predictions
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        all_predictions = []
        all_probabilities = []
        
        print("\nGenerating predictions...")
        with torch.no_grad():
            for batch in tqdm(test_loader, desc="Predicting"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                
                outputs = self.model(input_ids, attention_mask)
                logits = outputs['logits']
                
                # Get predictions and probabilities
                probs = torch.softmax(logits, dim=-1)
                predictions = torch.argmax(logits, dim=-1)
                
                all_predictions.extend(predictions.cpu().numpy())
                all_probabilities.extend(probs[:, 1].cpu().numpy())  # Probability of positive class
        
        # Add predictions to dataframe
        df_test['predicted_label'] = all_predictions
        df_test['breakpoint_probability'] = all_probabilities
        df_test['prediction'] = df_test['predicted_label'].map({0: 'Negative', 1: 'Positive'})
        
        # Save predictions
        print(f"\nSaving predictions to {output_path}...")
        df_test.to_csv(output_path, index=False, sep='\t')
        print("✅ Predictions saved!")
        
        return df_test

## 🎯 Step 6: Prepare Data

Load and prepare the training data from CSV files.

In [10]:
# Initialize data preparation
data_prep = BreakpointDataPreparation(columns=COLUMNS)

# Prepare data
df_combined = data_prep.prepare_data(
    positive_csv=POSITIVE_CSV,
    negative_csv=NEGATIVE_CSV
)

PREPARING TRAINING DATA
Loading datasets/fusion_gene_positive_bp_information_with_class_for_modeling.txt...


  Loaded 30745 samples
Loading datasets/fusion_gene_negative_bp_information_with_class_for_modeling.txt...
  Loaded 20000 samples
Merging 5' and 3' sequences...
  ⚠️ Warning: Found 15 sequences with non-ACGT characters
     These will be handled by the tokenizer
  Sequence length stats:
    Min: 20000 bp
    Max: 20000 bp
    Mean: 20000 bp
Merging 5' and 3' sequences...
  ⚠️ Warning: Found 116 sequences with non-ACGT characters
     These will be handled by the tokenizer
  Sequence length stats:
    Min: 20000 bp
    Max: 20000 bp
    Mean: 20000 bp

Combining datasets...

✅ Data preparation complete!
   Total samples: 50745
   Positive samples: 30745 (60.6%)
   Negative samples: 20000 (39.4%)


## 🏋️ Step 7: Initialize Pipeline and Prepare Datasets

Create the training pipeline and prepare train/validation datasets.

In [11]:
# Initialize pipeline
pipeline = SegmentNTTrainingPipeline(
    model_name=MODEL_NAME,
    output_dir=OUTPUT_DIR,
    max_length=MAX_LENGTH
)

# Prepare datasets
train_dataset, val_dataset = pipeline.prepare_datasets(
    df_combined, 
    val_split=VAL_SPLIT
)

Loading SegmentNT tokenizer...


  Tokenizer loaded!
  Vocabulary size: 4107
  Tokenization: 6-mer based

PREPARING DATASETS
Training samples: 40596
Validation samples: 10149
Using max token length: 3333 (for ~20000 bp sequences)
Using max token length: 3333 (for ~20000 bp sequences)

✅ Datasets prepared!


## 🚂 Step 8: Train the Model

Train SegmentNT on the breakpoint classification task.

**This will take some time depending on:**
- GPU type (A100 recommended)
- Dataset size
- Number of epochs

**Progress indicators:**
- Training loss (should decrease)
- Evaluation metrics per epoch
- Checkpoint saving

In [12]:
# Train the model
pipeline.train(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    num_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    weight_decay=WEIGHT_DECAY
)


INITIALIZING MODEL
Loading SegmentNT model: InstaDeepAI/segment_nt...


A new version of the following files was downloaded from https://huggingface.co/InstaDeepAI/segment_nt:
- segment_nt_config.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/InstaDeepAI/segment_nt:
- modeling_segment_nt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


ImportError: cannot import name 'find_pruneable_heads_and_indices' from 'transformers.modeling_utils' (/home/akp1/GeneAI/venv/lib/python3.12/site-packages/transformers/modeling_utils.py)

## 📊 Step 9: Evaluate Model Performance

Evaluate the trained model on the validation set.

In [ ]:
# Evaluate the model
eval_metrics = pipeline.evaluate()

print("\n" + "="*70)
print("FINAL EVALUATION RESULTS")
print("="*70)
print(f"\n📊 Model Performance:")
print(f"   Accuracy:  {eval_metrics['eval_accuracy']:.4f}")
print(f"   Precision: {eval_metrics['eval_precision']:.4f}")
print(f"   Recall:    {eval_metrics['eval_recall']:.4f}")
print(f"   F1 Score:  {eval_metrics['eval_f1']:.4f}")
print(f"   AUC-ROC:   {eval_metrics['eval_auc']:.4f}")

## 💾 Step 10: Save the Trained Model

Save the model for later use.

In [ ]:
# Save the model
pipeline.save_model()

## 🔮 Step 11: Make Predictions on Test Data

Load the test CSV file and make predictions using the trained model.

**Output:** CSV file with:
- All original columns
- `predicted_label` (0 or 1)
- `breakpoint_probability` (0.0 to 1.0)
- `prediction` ("Negative" or "Positive")

In [ ]:
print("="*70)
print("STEP 5: MAKE PREDICTIONS ON TEST DATA")
print("="*70)

predictions_df = pipeline.predict(
    test_csv_path=TEST_CSV,
    output_path="breakpoint_predictions.csv",
    batch_size=BATCH_SIZE
)

print("\n✅ Predictions complete!")

## 📋 Step 12: View Prediction Results

Display sample predictions and summary statistics.

In [ ]:
print("\n" + "="*70)
print("PREDICTION SUMMARY")
print("="*70)

# Summary statistics
print(f"\nTotal test samples: {len(predictions_df)}")
print(f"Predicted Positive: {(predictions_df['predicted_label'] == 1).sum()} ({(predictions_df['predicted_label'] == 1).sum() / len(predictions_df) * 100:.1f}%)")
print(f"Predicted Negative: {(predictions_df['predicted_label'] == 0).sum()} ({(predictions_df['predicted_label'] == 0).sum() / len(predictions_df) * 100:.1f}%)")
print(f"\nAverage breakpoint probability: {predictions_df['breakpoint_probability'].mean():.4f}")
print(f"Min probability: {predictions_df['breakpoint_probability'].min():.4f}")
print(f"Max probability: {predictions_df['breakpoint_probability'].max():.4f}")

# Show sample predictions
print("\n" + "="*70)
print("SAMPLE PREDICTIONS (first 5 rows)")
print("="*70)
display(predictions_df[['predicted_label', 'breakpoint_probability', 'prediction']].head())

# Probability distribution
print("\n" + "="*70)
print("PROBABILITY DISTRIBUTION")
print("="*70)
print(predictions_df['breakpoint_probability'].describe())

## 📊 (Optional) Step 13: Visualize Results

Create visualizations of the prediction results.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 4)

# Create subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 1. Prediction distribution
predictions_df['prediction'].value_counts().plot(kind='bar', ax=axes[0], color=['#FF6B6B', '#4ECDC4'])
axes[0].set_title('Prediction Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Prediction')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=0)

# 2. Probability distribution
axes[1].hist(predictions_df['breakpoint_probability'], bins=30, edgecolor='black', alpha=0.7, color='#95E1D3')
axes[1].set_title('Breakpoint Probability Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Probability')
axes[1].set_ylabel('Frequency')
axes[1].axvline(x=0.5, color='red', linestyle='--', linewidth=2, label='Threshold (0.5)')
axes[1].legend()

# 3. Box plot by prediction
predictions_df.boxplot(column='breakpoint_probability', by='prediction', ax=axes[2])
axes[2].set_title('Probability by Prediction', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Prediction')
axes[2].set_ylabel('Breakpoint Probability')
plt.suptitle('')  # Remove default title

plt.tight_layout()
plt.savefig('prediction_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Visualization saved as 'prediction_analysis.png'")

## 💾 Step 14: Save/Download Results

Download the results if you're using Google Colab.

In [ ]:
# For Google Colab: Uncomment these lines to download files
# from google.colab import files
# 
# # Download predictions
# files.download('breakpoint_predictions.csv')
# 
# # Download visualization (if created)
# files.download('prediction_analysis.png')
# 
# # Optionally download the model
# !zip -r segmentnt_model.zip segmentnt_breakpoint_model/final_model/
# files.download('segmentnt_model.zip')

print("\nFiles ready for download:")
print("  - breakpoint_predictions.csv")
print("  - prediction_analysis.png (if visualization was run)")
print(f"  - {OUTPUT_DIR}/final_model/ (trained model)")

---
# ✅ PIPELINE COMPLETE!

## What You've Accomplished:

1. ✅ Loaded and prepared FusionAI format data
2. ✅ Merged 5' and 3' sequences into 20kb inputs
3. ✅ Fine-tuned SegmentNT for binary classification
4. ✅ Evaluated model performance on validation data
5. ✅ Made predictions on test data
6. ✅ Saved results and visualizations

## Output Files:

- **`breakpoint_predictions.csv`** - Test predictions with probabilities
- **`segmentnt_breakpoint_model/final_model/`** - Trained model weights
- **`prediction_analysis.png`** - Visualization (if created)

## About SegmentNT:

SegmentNT is a powerful DNA foundation model that:
- Uses Nucleotide Transformer (500M parameters) as backbone
- Employs U-Net segmentation architecture
- Trained on 14 different genomic element classes
- Provides single-nucleotide resolution predictions
- Can handle sequences up to 50kb (with rescaling)

## Next Steps:

1. **Analyze results** - Check prediction quality and metrics
2. **Adjust hyperparameters** - Try different learning rates, batch sizes, or epochs
3. **Use the model** - Apply to new data using the saved model
4. **Experiment with sequence length** - Try longer sequences (up to 30kb or 50kb with rescaling)
5. **Compare with HyenaDNA** - Evaluate performance differences

---

## 🔄 To Use the Trained Model Later:

```python
# Load the pipeline
pipeline = SegmentNTTrainingPipeline(
    model_name=MODEL_NAME,
    output_dir=OUTPUT_DIR,
    max_length=MAX_LENGTH
)

# Make predictions on new data
new_predictions = pipeline.predict(
    test_csv_path="new_test_data.csv",
    model_path="./segmentnt_breakpoint_model/final_model",
    output_path="new_predictions.csv"
)
```

---

## 📚 References:

- **SegmentNT Paper:** [Segmenting the genome at single-nucleotide resolution with DNA foundation models](https://www.biorxiv.org/content/biorxiv/early/2024/03/15/2024.03.14.584712.full.pdf)
- **Model Hub:** [InstaDeepAI/segment_nt](https://huggingface.co/InstaDeepAI/segment_nt)
- **GitHub:** [Nucleotide Transformer Repository](https://github.com/instadeepai/nucleotide-transformer)

---

**Questions or issues?** Check the comments in each cell or refer to the SegmentNT documentation.

**Happy predicting! 🧬🚀**